# Donchian Breakout Managed Strategy (1D/4H/1H)

Hypothesis: in a confirmed uptrend (Close > SMA200 and SMA50 > SMA200), breakouts above the Donchian channel high that align with momentum and volume confirmation can capture strong continuation moves.


## Indicators
- SMA200 (long-term trend filter)
- SMA50 (mid-term trend filter)
- Donchian channel high/low (20 bars) for breakout levels
- RSI(14) momentum filter
- Volume SMA(20) confirmation

Exits are managed by ManagedStrategy risk controls (SL / TP / trailing).


In [2]:
from pathlib import Path
import sys

def find_project_root(start=None):
    start = Path.cwd() if start is None else Path(start)
    for candidate in [start] + list(start.parents):
        if (candidate / 'utils').exists() and (candidate / 'backtesting').exists():
            return candidate
    return start

PROJECT_ROOT = find_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print('Project root:', PROJECT_ROOT)


Project root: c:\Users\saill\Desktop\t_project


In [3]:
# Notebook configuration (edit as needed)
TICKER = 'SPY'
START_DATE = '2015-01-01'
END_DATE = '2025-01-01'

TIMEFRAME_DAILY = '1d'
INTRADAY_ENABLED = False
TIMEFRAME_INTRADAY = '1h'
RESAMPLE_TO = '4h'  # resample intraday feed to this timeframe

RISK_PCT = 0.02  # 2% risk per trade
DEFAULT_STOP_DISTANCE = 0.03  # used by FixedFractionalSizer

DONCHIAN_PERIOD = 20
RSI_PERIOD = 14
VOL_MA_PERIOD = 20


In [4]:
import pandas as pd

from utils.data_manager import DataManager
from utils.data_processor import resample_data

dm = DataManager()
df_daily = dm.get_data(TICKER, START_DATE, END_DATE, interval=TIMEFRAME_DAILY)
if INTRADAY_ENABLED:
    df_intraday = dm.get_data(TICKER, START_DATE, END_DATE, interval=TIMEFRAME_INTRADAY)

    if df_intraday is not None and not df_intraday.empty:
        df_4h = resample_data(df_intraday, rule=RESAMPLE_TO)
    else:
        df_4h = pd.DataFrame()
        print('Intraday data unavailable; continuing without 4H resample.')
    if not df_4h.empty:
        print(f'4H bars: {len(df_4h)}')

print(f'Daily bars: {len(df_daily)}')



2025-11-03 21:01:36 - utils.data_manager - INFO - DataManager initialisé. Cache: C:\Users\saill\Desktop\t_project\data\cache. Timezone: Europe/Paris
2025-11-03 21:01:36 - utils.data_manager - INFO - Cache insuffisant pour SPY (demandé: 2015-01-01 à 2025-01-01). Re-téléchargement.
2025-11-03 21:01:36 - utils.data_manager - INFO - Téléchargement pour SPY (plage par défaut : 2015-01-01 à 2025-11-03)...
2025-11-03 21:01:36 - utils.data_manager - INFO - Téléchargement de SPY (2015-01-01 à 2025-11-03, 1d)...
2025-11-03 21:01:37 - utils.data_manager - INFO - Données téléchargées avec succès pour SPY (2725 lignes).
2025-11-03 21:01:37 - utils.data_manager - INFO - [OK] Données prêtes pour SPY (2516 lignes de 2015-01-01 à 2025-01-01).


Daily bars: 2516


In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_plot = df_daily.copy()
try:
    import pandas_ta as ta
    df_plot['SMA50'] = ta.sma(df_plot['close'], length=50)
    df_plot['SMA200'] = ta.sma(df_plot['close'], length=200)
    df_plot['DonchianHigh'] = ta.highest(df_plot['high'], length=DONCHIAN_PERIOD).shift(1)
    df_plot['DonchianLow'] = ta.lowest(df_plot['low'], length=DONCHIAN_PERIOD).shift(1)
    df_plot['RSI14'] = ta.rsi(df_plot['close'], length=RSI_PERIOD)
    if 'volume' in df_plot.columns:
        df_plot['VolumeMA'] = ta.sma(df_plot['volume'], length=VOL_MA_PERIOD)
except Exception:
    df_plot['SMA50'] = df_plot['close'].rolling(50).mean()
    df_plot['SMA200'] = df_plot['close'].rolling(200).mean()
    df_plot['DonchianHigh'] = df_plot['high'].rolling(DONCHIAN_PERIOD).max().shift(1)
    df_plot['DonchianLow'] = df_plot['low'].rolling(DONCHIAN_PERIOD).min().shift(1)
    df_plot['RSI14'] = 50.0
    if 'volume' in df_plot.columns:
        df_plot['VolumeMA'] = df_plot['volume'].rolling(VOL_MA_PERIOD).mean()

if 'VolumeMA' not in df_plot.columns:
    df_plot['VolumeMA'] = 0.0

# Normalize timestamps for Plotly
if getattr(df_plot.index, 'tz', None) is not None:
    xs = df_plot.index.tz_convert('UTC').tz_localize(None)
else:
    xs = df_plot.index

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.03, row_heights=[0.6, 0.2, 0.2])
fig.add_trace(go.Candlestick(x=xs, open=df_plot['open'], high=df_plot['high'], low=df_plot['low'], close=df_plot['close'], name='OHLC'), row=1, col=1)
fig.add_trace(go.Scatter(x=xs, y=df_plot['SMA50'], mode='lines', name='SMA50'), row=1, col=1)
fig.add_trace(go.Scatter(x=xs, y=df_plot['SMA200'], mode='lines', name='SMA200'), row=1, col=1)
fig.add_trace(go.Scatter(x=xs, y=df_plot['DonchianHigh'], mode='lines', name='Donchian High', line=dict(color='orange')), row=1, col=1)
fig.add_trace(go.Scatter(x=xs, y=df_plot['DonchianLow'], mode='lines', name='Donchian Low', line=dict(color='teal')), row=1, col=1)
fig.add_trace(go.Scatter(x=xs, y=df_plot['RSI14'], mode='lines', name='RSI14', line=dict(color='purple')), row=2, col=1)
if 'volume' in df_plot.columns:
    fig.add_trace(go.Bar(x=xs, y=df_plot['volume'], name='Volume', marker_color='lightgrey'), row=3, col=1)
    if df_plot['VolumeMA'].dtype.kind in 'biufc':
        fig.add_trace(go.Scatter(x=xs, y=df_plot['VolumeMA'], mode='lines', name='Volume SMA', line=dict(color='darkgrey')), row=3, col=1)
fig.update_yaxes(title_text='Price', row=1, col=1)
fig.update_yaxes(title_text='RSI', row=2, col=1, range=[0, 100])
fig.update_yaxes(title_text='Volume', row=3, col=1)
fig.update_layout(title=f'{TICKER} Donchian breakout context', xaxis_rangeslider_visible=False, template='plotly_white')
fig.show()


In [6]:
import backtrader as bt

from backtesting.engine import BacktestEngine
from risk_management.position_sizing import FixedFractionalSizer
from strategies.implementations.donchian_breakout_managed_strategy import DonchianBreakoutManagedStrategy
from utils.config_loader import get_settings

engine = BacktestEngine()
engine.cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timereturn')
engine.cerebro.addanalyzer(bt.analyzers.Transactions, _name='transactions')

engine.add_sizer(FixedFractionalSizer, risk_pct=RISK_PCT, stop_distance=DEFAULT_STOP_DISTANCE)
engine.add_data(df_daily, name='data_1d')
if 'df_4h' in globals() and not df_4h.empty:
    engine.add_data(df_4h, name='data_4h')

engine.add_strategy(
    DonchianBreakoutManagedStrategy,
    trend_long_period=200,
    trend_mid_period=50,
    donchian_period=DONCHIAN_PERIOD,
    rsi_period=RSI_PERIOD,
    rsi_min=50,
    vol_ma_period=VOL_MA_PERIOD,
    require_volume_confirm=True,
    use_invalidation=True,
    reentry_cooldown_bars=5,
    atr_period=14,
)

results = engine.run()
strategy_instance = results[0]


2025-11-03 21:01:41 - backtesting.engine - INFO - Initialisation du BacktestEngine...
2025-11-03 21:01:41 - backtesting.engine - INFO - Capital initial du broker fixé à : 10,000.00
2025-11-03 21:01:41 - backtesting.engine - INFO - Commission (pourcentage) fixée à : 0.1000%
2025-11-03 21:01:41 - backtesting.engine - INFO - Slippage (pourcentage) fixé à : 0.0500%
2025-11-03 21:01:41 - backtesting.engine - INFO - Position Sizer 'FixedFractionalSizer' ajouté. Paramètres: (risk_pct=0.02, stop_distance=0.03)
2025-11-03 21:01:41 - backtesting.engine - INFO - Flux de données 'data_1d' ajouté. Période: 2015-01-02 à 2024-12-31.
2025-11-03 21:01:41 - backtesting.engine - INFO - Stratégie 'DonchianBreakoutManagedStrategy' ajoutée. Paramètres: (trend_long_period=200, trend_mid_period=50, donchian_period=20, rsi_period=14, rsi_min=50, vol_ma_period=20, require_volume_confirm=True, use_invalidation=True, reentry_cooldown_bars=5, atr_period=14)
2025-11-03 21:01:41 - backtesting.engine - INFO - --- DÉM

In [7]:
initial_capital = get_settings().get('backtest', {}).get('initial_capital', 10000.0)
try:
    from scripts.run_backtest import print_results
    print_results(results, initial_capital, df_daily)
except Exception as exc:
    final_value = strategy_instance.broker.getvalue()
    print(f'Final portfolio value: {final_value:,.2f}')
    print(f'(Detailed metrics unavailable: {exc})')



RÉSULTATS DU BACKTEST

📊 Période: 2015-01-02 à 2024-12-31
📊 Nombre de bougies: 2516

💰 PERFORMANCE
   Capital Initial:              10,000.00 €
   Capital Final:                11,641.48 €
   P&L:                           1,641.48 € (+16.41%)
   Retour Total:                     15.20%
   Retour Moyen (annuel):             0.01%

📈 TRADES
   Nombre Total:                        27
   Trades Gagnants:                     15
   Trades Perdants:                     12
   Win Rate:                        55.56%
   Gain Moyen:                      277.13 €
   Perte Moyenne:                  -209.62 €

📉 RISQUE
   Sharpe Ratio:           0.13919020863002254
   Max Drawdown:                    14.97%



In [8]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def _to_naive(dt):
    if dt is None:
        return None
    ts = pd.Timestamp(dt)
    if ts.tzinfo is not None:
        ts = ts.tz_convert('UTC').tz_localize(None)
    return ts

# Extract transactions (buys/sells) for markers
tx = strategy_instance.analyzers.transactions.get_analysis()
buy_times, buy_prices, sell_times, sell_prices = [], [], [], []
for k, v in (tx or {}).items():
    ts = _to_naive(k)
    for t in v:
        size = t[0] if len(t) > 0 else None
        price = t[1] if len(t) > 1 else None
        if size is None or size == 0:
            continue
        if size > 0:
            buy_times.append(ts); buy_prices.append(price)
        elif size < 0:
            sell_times.append(ts); sell_prices.append(price)

# Equity curve
time_returns = strategy_instance.analyzers.timereturn.get_analysis()
if time_returns:
    equity_series = pd.Series(time_returns).sort_index()
    equity_curve = initial_capital * (1 + equity_series).cumprod()
else:
    equity_curve = pd.Series(dtype='float64')

# Price timeline
if getattr(df_daily.index, 'tz', None) is not None:
    price_index = df_daily.index.tz_convert('UTC').tz_localize(None)
else:
    price_index = df_daily.index

fig_bt = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.7, 0.3])
fig_bt.add_trace(go.Candlestick(x=price_index, open=df_daily['open'], high=df_daily['high'], low=df_daily['low'], close=df_daily['close'], name='OHLC'), row=1, col=1)
if 'SMA50' in df_plot.columns:
    fig_bt.add_trace(go.Scatter(x=price_index, y=df_plot['SMA50'], mode='lines', name='SMA50'), row=1, col=1)
if 'SMA200' in df_plot.columns:
    fig_bt.add_trace(go.Scatter(x=price_index, y=df_plot['SMA200'], mode='lines', name='SMA200'), row=1, col=1)
if 'DonchianHigh' in df_plot.columns:
    fig_bt.add_trace(go.Scatter(x=price_index, y=df_plot['DonchianHigh'], mode='lines', name='Donchian High', line=dict(color='orange')), row=1, col=1)
if 'DonchianLow' in df_plot.columns:
    fig_bt.add_trace(go.Scatter(x=price_index, y=df_plot['DonchianLow'], mode='lines', name='Donchian Low', line=dict(color='teal')), row=1, col=1)

if buy_times:
    fig_bt.add_trace(go.Scatter(x=buy_times, y=buy_prices, mode='markers', name='Buy', marker=dict(symbol='triangle-up', color='green', size=10)), row=1, col=1)
if sell_times:
    fig_bt.add_trace(go.Scatter(x=sell_times, y=sell_prices, mode='markers', name='Sell', marker=dict(symbol='triangle-down', color='red', size=10)), row=1, col=1)

if not equity_curve.empty:
    eq_index = equity_curve.index
    if getattr(eq_index, 'tz', None) is not None:
        eq_index = eq_index.tz_convert('UTC').tz_localize(None)
    fig_bt.add_trace(go.Scatter(x=eq_index, y=equity_curve, mode='lines', name='Equity Curve'), row=2, col=1)
else:
    fig_bt.add_trace(go.Scatter(x=[price_index.min()], y=[initial_capital], mode='lines', name='Equity Curve'), row=2, col=1)

fig_bt.update_yaxes(title_text='Price', row=1, col=1)
fig_bt.update_yaxes(title_text='Equity', row=2, col=1)
fig_bt.update_layout(title='Donchian breakout backtest review', template='plotly_white', xaxis_rangeslider_visible=False)
fig_bt.show()
